<a href="https://colab.research.google.com/github/aroy1856/mlproject/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk

In [2]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [3]:
from nltk.corpus import gutenberg
import  pandas as pd

## load the dataset
data=gutenberg.raw('shakespeare-hamlet.txt')
## save to a file
with open('hamlet.txt','w') as file:
    file.write(data)

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
with open('hamlet.txt','r') as file:
    data=file.read().lower()

tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [6]:
words = len(tokenizer.word_index) + 1

In [7]:
tokenizer.texts_to_sequences(data.split('\n')[0])[0]

[]

In [8]:
## create inoput sequences
input_sequences=[]
for line in data.split('\n'):
    # print(line)
    token_list=tokenizer.texts_to_sequences([line])[0]
    # print(token_list)
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        # print(n_gram_sequence)
        input_sequences.append(n_gram_sequence)

In [9]:
max_len = max([len(x) for x in input_sequences])

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split

In [11]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [12]:
x,y=input_sequences[:,:-1],input_sequences[:,-1]

In [13]:
import tensorflow as tf

In [14]:
y = tf.keras.utils.to_categorical(y, num_classes=words)

In [15]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout, GRU

In [21]:
model=Sequential()
model.add(Embedding(words,100,input_length=max_len-1))
model.add(GRU(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100, return_sequences=True))
model.add(GRU(80))
# model.add(Dense(100,activation="relu"))
model.add(Dense(words,activation="softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [22]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Define early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [24]:
model.fit(x_train,y_train,epochs=150,validation_data=(x_test,y_test),verbose=1)

Epoch 1/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - accuracy: 0.0303 - loss: 7.1446 - val_accuracy: 0.0231 - val_loss: 6.8690
Epoch 2/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0283 - loss: 6.5871 - val_accuracy: 0.0402 - val_loss: 6.8759
Epoch 3/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.0373 - loss: 6.4582 - val_accuracy: 0.0523 - val_loss: 6.8413
Epoch 4/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.0494 - loss: 6.2387 - val_accuracy: 0.0655 - val_loss: 6.8137
Epoch 5/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.0668 - loss: 5.9650 - val_accuracy: 0.0694 - val_loss: 6.7741
Epoch 6/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.0781 - loss: 5.7582 - val_accuracy: 0.0746 - val_loss: 6.8195
Epoch 7/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.0895 - loss: 5.5106 - val_accuracy: 0.0742 - val_loss: 6.8822
Epoch 8/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.0939 - loss: 5.32

In [25]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 13, 150)        │       113,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 13, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ (None, 13, 100)        │        75,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_5 (GRU)                     │ (None, 80)             │        43,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4818)           │       390,258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,314,216 (12.64 MB)

 Trainable params: 1,104,738 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,209,478 (8.43 MB)